In [1]:
from llm.LLMModel import *
from ner.llm_ner.prompt_techniques.pt_abstract import PromptTechnique
from ner.llm_ner.prompt_techniques.pt_discussion import PT_OutputList
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_gpt_ner import PT_GPT_NER
from ner.llm_ner.prompt_techniques.pt_wrapper import PT_Wrapper
from ner.llm_ner.few_shots_techniques import *
from ner.llm_ner.ResultInstance import load_all_results, load_result
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%reload_ext autoreload
%autoreload 2

In [4]:
df_results = load_all_results(root_directory = "ner/saves/results/conll2003_cleaned/")
df_results[0].sort_values(['f1_mean', 'prompt_technique', 'nb_few_shots'], ascending = False).head(10)

,f1_mean,f1_conf_inter,model,noshots,prompt_technique,few_shot_tecnique,nb_few_shots,precision,plus_plus,verifier,len_data_train,len_data_test,nb_test_run,confidence_interval,distribution_used,precision_mean,precision_conf_inter,recall_mean,recall_conf_inter
4,0.923,"(0.877, 0.969)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,entity,5,False,False,None,1538,50,3,0.95,Student,0.919,"(0.831, 1.007)",0.873,"(0.800, 0.946)"
5,0.912,"(0.836, 0.988)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,sentence,5,True,False,None,1538,50,3,0.95,Student,0.924,"(0.817, 1.031)",0.855,"(0.740, 0.970)"
10,0.890,"(0.846, 0.935)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,entity,5,True,False,None,1538,50,3,0.95,Student,0.875,"(0.744, 1.006)",0.821,"(0.757, 0.885)"
3,0.890,"(0.845, 0.936)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,random,5,True,False,None,1538,50,3,0.95,Student,0.870,"(0.813, 0.927)",0.823,"(0.750, 0.895)"
6,0.888,"(0.840, 0.935)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,sentence,5,False,False,None,1538,50,3,0.95,Student,0.875,"(0.839, 0.911)",0.827,"(0.772, 0.883)"
7,0.886,"(0.844, 0.928)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,random,5,False,False,None,1538,50,3,0.95,Student,0.865,"(0.822, 0.908)",0.825,"(0.770, 0.880)"
12,0.867,"(0.843, 0.891)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,no-shots,5,True,False,None,1538,50,3,0.95,Student,0.818,"(0.778, 0.859)",0.800,"(0.749, 0.851)"
114,0.854,"(0.782, 0.926)",mistral-7b-v0.1-ft-raw-conll2003-2000-Q5_0,False,wrapper,sentence,10,300,True,None,1488,100,3,0.95,Student,0.860,"(0.784, 0.936)",0.851,"(0.785, 0.917)"
115,0.844,"(0.769, 0.919)",mistral-7b-v0.1-ft-raw-conll2003-2000-Q5_0,False,wrapper,sentence,3,300,True,None,1488,100,3,0.95,Student,0.855,"(0.785, 0.924)",0.837,"(0.762, 0.912)"
11,0.841,"(0.832, 0.850)",mistral-7b-v0.1-ft-discussion-2000-Q5_0,False,discussion,random,5,True,False,None,1538,50,10,0.95,Student,0.872,"(0.862, 0.882)",0.816,"(0.807, 0.826)"


In [ ]:
df_results[1][7].res_insts[0].analyse_results()

In [ ]:
df_results = load_all_results(root_directory = "ner/saves/results/ontonote5/")
df_results = df_results[df_results['prompt_technique'] == 'discussion']
df_results = df_results[df_results['few_shot_tecnique'] == 'sentence']
df_results.sort_values(['nb_few_shots', 'model'])

In [ ]:
df_to_show = df_results[~df_results['model'].str.contains('ft')][['f1_mean', 'f1_conf_inter', 'prompt_technique',
       'few_shot_tecnique', 'nb_few_shots', 'precision', 'plus_plus']]

df_res = df_to_show[df_to_show['precision'] == '300']
df_res['tech_name'] = df_res.apply(lambda row :f"With {row['nb_few_shots']} few_shots {'and ++' if row['plus_plus']  else ''}", axis = 1)
df_res

## Showing prompts 

In [ ]:
LLMModel.show_prompts()

## Looking at results of confidence with the tagger

In [ ]:
from ner.utils import load

res = load ("ner/saves/results/conll2003_cleaned/mistral-7b-v0.1/multi_prompt-get-entities-tagger/sentence_3_None_1538_50_1_False.pkl")

cm, f1, precision, recall, y_true, y_pred, y_conf= res.res_insts[0].get_scores(with_y = True)
conf = []
for sent in res.res_insts[0].results:
    conf.extend([s[2] for s in sent])

len(y_conf), len(y_true), len(y_pred)
# [i for i, tag in enumerate(y_pred) if tag == 'None']
right = [y_true[i] == y_pred[i] for i in range(len(y_true))]
rand = [i for i in range(len(right))]

In [ ]:
from collections import Counter

# Example list
my_list = list(zip(y_conf, right))

# Use Counter to count occurrences
entry_counts = Counter(sorted(my_list))

# Print the counts
for entry, count in entry_counts.items():
    print(f'{entry}: {count} times')

In [ ]:
import seaborn as sns
df_conf = pd.DataFrame(np.array([right, y_conf]).T, columns = ['Right', 'Confidence'])

df_conf = pd.DataFrame(df_conf.groupby(['Right', 'Confidence'])['Right'].count()).rename(columns = {'Right' : "count"}).reset_index()
sns.set(rc={'figure.figsize':(10,10)})
order = ['high', 'medium-high', 'medium', 'medium-low', 'low']
sns.catplot(data=df_conf, x="Confidence", y="count", hue="Right", kind="bar",aspect=1.5, order = order)


## Analysing specific result

In [ ]:
model  = "mistral-7b-v0.1"
result = load_result(model, PT_OutputList.name(), FST_Sentence.name(), nb_few_shots= 10, with_precision = "10_True")

for r in result.res_insts :
    r.show_cm()
    r.analyse_results()

In [ ]:
r.f1

In [ ]:
from ner.Datasets.Conll2003Dataset import Conll2003Dataset
from ner.Datasets.MyDataset import MyDataset
data = MyDataset.my_load_dataset(dataset=Conll2003Dataset, split = 'test', cleaned= True)
[(d['text'], d['spans']) for d in data if str(d['spans']) == """[['Switzerland', 'LOC'], ["Alfonse D'Amato", 'PER'], ['U.S. Senate Banking Committee', 'ORG'], ['Poland', 'LOC'], ['Polish', 'MISC'], ['Swiss', 'MISC'], ['Poland', 'LOC']]"""]

In [ ]:
[(d['text'], d['spans']) for d in data if 'Hansenne' in d['text']]